In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import Ridge, LogisticRegression, SGDClassifier, SGDRegressor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate

## Combining multiple tables (15 min)

- Take a look at the [Formula 1 race data set](https://www.kaggle.com/cjgdev/formula-1-race-data-19502017) from Kaggle. 
- The dataset contains **multiple CSV files**.
- Let's read in one of them:

In [3]:
racing_results_df = pd.read_csv("data/formula-1-race-data-19502017/results.csv", index_col=0)
racing_results_df

,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
resultId,,,,,,,,,,,,,,,,,
1,18,1,1,22.0,1,1.0,1,1,10.0,58,34:50.6,5690616.0,39.0,2.0,01:27.5,218.3,1
2,18,2,2,3.0,5,2.0,2,2,8.0,58,5.478,5696094.0,41.0,3.0,01:27.7,217.586,1
3,18,3,3,7.0,7,3.0,3,3,6.0,58,8.163,5698779.0,41.0,5.0,01:28.1,216.719,1
4,18,4,4,5.0,11,4.0,4,4,5.0,58,17.181,5707797.0,58.0,7.0,01:28.6,215.464,1
5,18,5,1,23.0,3,5.0,5,5,4.0,58,18.014,5708630.0,43.0,1.0,01:27.4,218.385,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23777,988,842,5,10.0,17,16.0,16,16,0.0,54,NaN,NaN,33.0,16.0,01:43.8,192.542,11
23778,988,828,15,9.0,19,17.0,17,17,0.0,54,NaN,NaN,36.0,15.0,01:43.6,193.057,11
23779,988,840,3,18.0,15,18.0,18,18,0.0,54,NaN,NaN,52.0,6.0,01:42.3,195.402,11


- Let's say we want to predict the `milliseconds` column, namely the total length of time it takes a driver to finish a race. 
- In that case, we should not have access to most of these other columns. 
- But we would have the `raceId` and `driverId`:

In [4]:
racing_results_df_subset = racing_results_df.dropna(subset=["milliseconds"])[['raceId', 'driverId', 'milliseconds']]
racing_results_df_subset

,raceId,driverId,milliseconds
resultId,,,
1,18,1,5690616.0
2,18,2,5696094.0
3,18,3,5698779.0
4,18,4,5707797.0
5,18,5,5708630.0
...,...,...,...
23765,988,8,5699448.0
23766,988,830,5700331.0
23767,988,807,5739775.0


In [5]:
racing_train, racing_test = train_test_split(racing_results_df_subset, random_state=123)

We can try some baselines based on the ID values themselves, although:

In [6]:
pd.DataFrame(cross_validate(DummyRegressor(), racing_train[["raceId", "driverId"]], 
                            racing_train["milliseconds"], return_train_score=True, cv=20)).mean()

fit_time       0.001468
score_time     0.000463
test_score    -0.002014
train_score    0.000000
dtype: float64

In [7]:
lr = make_pipeline(OneHotEncoder(handle_unknown='ignore'), Ridge())
pd.DataFrame(cross_validate(lr, racing_train[["raceId", "driverId"]], 
                            racing_train["milliseconds"], return_train_score=True, cv=20)).mean()

fit_time       0.009258
score_time     0.001341
test_score     0.949340
train_score    0.981948
dtype: float64

That is actually pretty good *facepalm*.

But what if we want to predict for a _new_ race track or a _new_ driver??

- Enter the other tables:

In [8]:
racing_drivers_df = pd.read_csv("data/formula-1-race-data-19502017/drivers.csv", 
                                encoding='latin-1', index_col=0,
                               parse_dates=['dob'])
racing_drivers_df

,driverRef,number,code,forename,surname,dob,nationality,url
driverId,,,,,,,,
1,hamilton,44.0,HAM,Lewis,Hamilton,1985-07-01,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
2,heidfeld,NaN,HEI,Nick,Heidfeld,1977-10-05,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
3,rosberg,6.0,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
4,alonso,14.0,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
5,kovalainen,NaN,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen
...,...,...,...,...,...,...,...,...
838,vandoorne,2.0,VAN,Stoffel,Vandoorne,1992-03-26,Belgian,http://en.wikipedia.org/wiki/Stoffel_Vandoorne
839,ocon,31.0,OCO,Esteban,Ocon,1996-09-17,French,http://en.wikipedia.org/wiki/Esteban_Ocon
840,stroll,18.0,STR,Lance,Stroll,1998-10-29,Canadian,http://en.wikipedia.org/wiki/Lance_Stroll


- Can we use the driver's nationality and age as features?
- `pd.merge` can take care of this for us.

In [9]:
merged_df = pd.merge(racing_results_df_subset, racing_drivers_df, on="driverId")
merged_df

,raceId,driverId,milliseconds,driverRef,number,code,forename,surname,dob,nationality,url
0,18,1,5690616.0,hamilton,44.0,HAM,Lewis,Hamilton,1985-07-01,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,19,1,5525103.0,hamilton,44.0,HAM,Lewis,Hamilton,1985-07-01,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
2,21,1,5903238.0,hamilton,44.0,HAM,Lewis,Hamilton,1985-07-01,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
3,22,1,5213230.0,hamilton,44.0,HAM,Lewis,Hamilton,1985-07-01,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
4,23,1,7242742.0,hamilton,44.0,HAM,Lewis,Hamilton,1985-07-01,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
...,...,...,...,...,...,...,...,...,...,...,...
5998,981,840,4606468.0,stroll,18.0,STR,Lance,Stroll,1998-10-29,Canadian,http://en.wikipedia.org/wiki/Lance_Stroll
5999,982,840,7445240.0,stroll,18.0,STR,Lance,Stroll,1998-10-29,Canadian,http://en.wikipedia.org/wiki/Lance_Stroll
6000,976,838,7527733.0,vandoorne,2.0,VAN,Stoffel,Vandoorne,1992-03-26,Belgian,http://en.wikipedia.org/wiki/Stoffel_Vandoorne
6001,980,838,5139898.0,vandoorne,2.0,VAN,Stoffel,Vandoorne,1992-03-26,Belgian,http://en.wikipedia.org/wiki/Stoffel_Vandoorne


- The `on` keyword told it which column to use to match up the rows of the two dataframes.
- Note that the first 5 rows have the same `driverId`, so they pulled the same data from `racing_drivers_df`.
- Now we could keep only the columns we plan to encode:

In [10]:
merged_df_subset = merged_df[['raceId', 'driverId', 'milliseconds', 'dob', 'nationality']]
merged_df_subset

,raceId,driverId,milliseconds,dob,nationality
0,18,1,5690616.0,1985-07-01,British
1,19,1,5525103.0,1985-07-01,British
2,21,1,5903238.0,1985-07-01,British
3,22,1,5213230.0,1985-07-01,British
4,23,1,7242742.0,1985-07-01,British
...,...,...,...,...,...
5998,981,840,4606468.0,1998-10-29,Canadian
5999,982,840,7445240.0,1998-10-29,Canadian
6000,976,838,7527733.0,1992-03-26,Belgian
6001,980,838,5139898.0,1992-03-26,Belgian


Now we can process the `dob` column to get age:

In [11]:
ages = (pd.Timestamp.now() - merged_df_subset["dob"]).apply(lambda x: x.total_seconds()/3600/24/365)
merged_df_age = merged_df_subset.assign(age=ages)
merged_df_age

,raceId,driverId,milliseconds,dob,nationality,age
0,18,1,5690616.0,1985-07-01,British,35.434304
1,19,1,5525103.0,1985-07-01,British,35.434304
2,21,1,5903238.0,1985-07-01,British,35.434304
3,22,1,5213230.0,1985-07-01,British,35.434304
4,23,1,7242742.0,1985-07-01,British,35.434304
...,...,...,...,...,...,...
5998,981,840,4606468.0,1998-10-29,Canadian,22.097318
5999,982,840,7445240.0,1998-10-29,Canadian,22.097318
6000,976,838,7527733.0,1992-03-26,Belgian,28.694578
6001,980,838,5139898.0,1992-03-26,Belgian,28.694578


- So far we got information for each driver.
- Likewise, we can get information about the races, and use those as well.

In [12]:
racing_races_df = pd.read_csv("data/formula-1-race-data-19502017/races.csv", encoding='latin-1', index_col=0)
racing_races_df

,year,round,circuitId,name,date,time,url
raceId,,,,,,,
1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...
...,...,...,...,...,...,...,...
1005,2018,17,22,Japanese Grand Prix,2018-10-07,05:00:00,http://en.wikipedia.org/wiki/2018_Japanese_Gra...
1006,2018,18,69,United States Grand Prix,2018-10-21,19:00:00,http://en.wikipedia.org/wiki/2018_United_State...
1007,2018,19,32,Mexican Grand Prix,2018-10-28,19:00:00,http://en.wikipedia.org/wiki/2018_Mexican_Gran...


Etc.

- For those who have taken CPSC 304 or some have other database training, you'll recognize this type of multi-table situation, with foreign keys connecting the tables.
- `pd.merge` supports several types of joins, see the documentation:

In [13]:
merged_df_full = pd.merge(merged_df_age, racing_races_df, on="raceId")
merged_df_full.head()

,raceId,driverId,milliseconds,dob,nationality,age,year,round,circuitId,name,date,time,url
0,18,1,5690616.0,1985-07-01,British,35.434304,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...
1,18,2,5696094.0,1977-10-05,German,43.176770,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...
2,18,3,5698779.0,1985-06-27,German,35.445263,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...
3,18,4,5707797.0,1981-07-29,Spanish,39.360332,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...
4,18,5,5708630.0,1981-10-19,Finnish,39.135674,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...


Maybe we just take the circuit ID:

In [14]:
merged_df_final = merged_df_full.drop(
    columns=["dob", "name", "date", "time", "url", "raceId", "driverId"])
merged_df_final

,milliseconds,nationality,age,year,round,circuitId
0,5690616.0,British,35.434304,2008,1,1
1,5696094.0,German,43.176770,2008,1,1
2,5698779.0,German,35.445263,2008,1,1
3,5707797.0,Spanish,39.360332,2008,1,1
4,5708630.0,Finnish,39.135674,2008,1,1
...,...,...,...,...,...,...
5998,10864600.0,Italian,102.448003,1953,1,25
5999,10110300.0,Italian,102.448003,1953,4,13
6000,10278500.0,Italian,111.612386,1953,4,13
6001,10800000.0,Italian,102.448003,1952,4,53


In [15]:
categorical_features = ["nationality", "round", "circuitId"]
numeric_features = ["age", "year"]

preprocessing = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore'), categorical_features),
    (StandardScaler(), numeric_features)
)
lr = make_pipeline(preprocessing, Ridge())
# pd.DataFrame(cross_validate(lr, merged_df_final.drop(columns=["milliseconds"]), 
#                             merged_df_final["milliseconds"], return_train_score=True, cv=20)).mean()

In [16]:
lr.fit(merged_df_final.drop(columns=["milliseconds"]), merged_df_final["milliseconds"]);

This new model can be used even for new drivers and races because it's based on features we got from these other tables.

In [17]:
# preprocessing.fit(merged_df_final)
ohe_columns = list(preprocessing.named_transformers_['onehotencoder'].get_feature_names(categorical_features))
new_columns = ohe_columns + numeric_features
lr_coefs = pd.DataFrame(data=lr[1].coef_, index=new_columns, columns=["Coefficient"])
#lr_coefs